# Decision Tree Calssifier:

In [4]:
#imports
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [5]:
#reading csv files from download
train_metadata = pd.read_csv('../../../isic-2024-challenge/train-metadata.csv')
test_metadata = pd.read_csv('../../../isic-2024-challenge/test-metadata.csv')
# print("\nTrain Metadata:\n", train_metadata.head())
# print("\nTest Metadata:\n", test_metadata.head())

/var/folders/q7/0fmp_ns55qzft8fc_lhg8g100000gn/T/ipykernel_1596/629991581.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv('../../../isic-2024-challenge/train-metadata.csv')


In [7]:
# "skin_cancer_decisiontree" is instantiated as an "object" of DecisionTreeClassifier "class". 

skin_cancer_decisiontree = DecisionTreeClassifier(criterion='entropy', max_depth=None,)

In [8]:
#use the method "fit" of the objects "my_decisiontree" along with training dataset and labels to train the model.

skin_cancer_decisiontree.fit(X_train, y_train)

NameError: name 'X_train' is not defined